# Medical Orders Data Exploration
Welcome! This notebook walks through an **exploratory analysis** of the `mma_fe_innovation.mma.medical_orders_silver` dataset, which contains medical order information from wearable device retailers.  

## Objectives
The goal of this exercise is to:

1. **Understand the dataset**
   - Examine structure, data types, key columns, and general content.
   - Identify missing values, anomalies, and potential outliers.

2. **Assess data quality**
   - Ensure that the dataset is reliable for analysis and AI modeling.
   - Detect and flag unusual values that could affect downstream results.

3. **Identify patterns and relationships**
   - Aggregate orders by retailer, device, and combinations.
   - Highlight popular products and trends that can inform business decisions.

4. **Set the stage for AI & predictive analytics**
   - Prepare a clean, well-understood dataset for building predictive models.
   - Support tasks like high-demand device prediction, anomaly detection, and recommendation systems.

> **Why this matters:** A thorough exploration builds a solid foundation for AI insights. It ensures that any models or predictions are grounded in a deep understanding of the real-world data, reducing risk and improving reliability.

## How to use this notebook
- Run each section sequentially.  
- Use the visualizations and summaries to guide understanding.  
- Identify any issues that need cleaning or further investigation before modeling.

## 1. Load the Table & Peek at the Data

In [0]:
from pyspark.sql import functions as F

# load
df = spark.table("mma_fe_innovation.mma.medical_orders_silver")

# schema + quick peek
df.printSchema()
display(df.limit(200))


##2. Dataset Overview & Null Analysis

In [0]:
from pyspark.sql import functions as F

summary = df.select(
    F.count("*").alias("rows"),
    F.countDistinct("order_id").alias("distinct_order_id"),
    F.countDistinct("retailer_name").alias("distinct_retailer"),
    F.countDistinct("device_name").alias("distinct_device"),
    F.min("order_date").alias("min_order_date"),
    F.max("order_date").alias("max_order_date")
)
display(summary)

nulls = df.select([F.sum(F.when(F.col(c).isNull() | (F.col(c) == ""), 1).otherwise(0)).alias(c + "_nulls")
                   for c in df.columns])
display(nulls)
from pyspark.sql import functions as F

summary = df.select(
    F.count("*").alias("rows"),
    F.countDistinct("order_id").alias("distinct_order_id"),
    F.countDistinct("retailer_name").alias("distinct_retailer"),
    F.countDistinct("device_name").alias("distinct_device"),
    F.min("order_date").alias("min_order_date"),
    F.max("order_date").alias("max_order_date")
)
display(summary)

nulls = df.select([F.sum(F.when(F.col(c).isNull() | (F.col(c) == ""), 1).otherwise(0)).alias(c + "_nulls")
                   for c in df.columns])
display(nulls)


##3. Timestamp Handling & Derived Date Columns
Dates are central to exploring time-series behavior. Here we:
 - Parse order_date as a timestamp
 - Derive year, month, and week columns
 - Flag any rows with invalid/unparsable dates

In [0]:
from pyspark.sql import functions as F

df_time = (
    df.withColumn("order_ts", F.to_timestamp("order_date"))
      .withColumn("order_date_only", F.to_date("order_ts"))
      .withColumn("order_year", F.year("order_ts"))
      .withColumn("order_month", F.date_format("order_ts", "yyyy-MM"))
      .withColumn("order_week", F.concat_ws(
          "-",
          F.year("order_ts"),
          F.format_string("%02d", F.weekofyear("order_ts"))
      ))
)

bad_dates = df_time.filter(F.col("order_ts").isNull()).limit(50)
print("Unparsed dates:", df_time.filter(F.col("order_ts").isNull()).count())
display(bad_dates)

#4. Quantity Distribution & Outliers

In [0]:
display(df.select("quantity").describe())

quantiles = df.approxQuantile("quantity", [0.01,0.05,0.25,0.5,0.75,0.95,0.99,1.0], 0.001)
print(dict(zip(["p1","p5","p25","p50","p75","p95","p99","max"], quantiles)))

non_positive = df.filter(F.col("quantity") <= 0).limit(50)
print("Non-positive quantities:", df.filter(F.col("quantity") <= 0).count())
display(non_positive)

display(df.orderBy(F.desc("quantity")).limit(50))


#5. Retailer and Device Popularity

In [0]:
retailer_stats = df.groupBy("retailer_name").agg(
    F.count("*").alias("rows"),
    F.countDistinct("order_id").alias("distinct_orders"),
    F.sum("quantity").alias("total_quantity")
).orderBy(F.desc("total_quantity"))

device_stats = df.groupBy("device_name").agg(
    F.count("*").alias("rows"),
    F.countDistinct("order_id").alias("distinct_orders"),
    F.sum("quantity").alias("total_quantity")
).orderBy(F.desc("total_quantity"))

display(retailer_stats.limit(25))
display(device_stats.limit(25))


#6. Retailer and Device Combinations

In [0]:
from pyspark.sql.window import Window

combo = df.groupBy("retailer_name", "device_name").agg(
    F.count("*").alias("rows"),
    F.sum("quantity").alias("total_qty")
).orderBy(F.desc("total_qty"))
display(combo.limit(50))

w = Window.partitionBy("retailer_name").orderBy(F.desc("total_qty"))
top_per_retailer = combo.withColumn("rank", F.row_number().over(w)).filter(F.col("rank") <= 3).orderBy("retailer_name", "rank")
display(top_per_retailer)
